In [1]:
import sys
#sys.append("..")
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ""
import numpy as np

def mA(y_pred, y_true):
    M = len(y_pred)
    L = len(y_pred[0])
    res = 0
    for i in range(L):
        P = sum(y_true[:, i])
        N = M - P
        TP = sum(y_pred[:, i]*y_true[:, i])
        TN = list(y_pred[:, i]+y_true[:, i] == 0).count(True)
        #print(P,',', N,',', TP,',', TN)
        if P != 0:
            res += TP/P + TN/N
        else:
            res += TN/N
    return res / (2*L)

def acc(y_pred, y_true):
    M = len(y_pred)
    M_ = 0
    res = 0
    for i in range(M):
        #print(np.shape(y_pred[i]*y_true[i]))
        if sum(y_pred[i])+sum(y_true[i])-sum(y_pred[i]*y_true[i]) != 0:
            res += sum(y_pred[i]*y_true[i]) / (sum(y_pred[i])+sum(y_true[i])-sum(y_pred[i]*y_true[i]))
            M_ += 1
    return res / M_

def prec(y_pred, y_true):
    M = len(y_pred)
    M_ = 0
    res = 0
    for i in range(M):
        if sum(y_pred[i]) != 0:
            res += sum(y_pred[i]*y_true[i]) / sum(y_pred[i])
            M_ += 1
    if M_ == 0:
        return 0
    return res / M_

def rec(y_pred, y_true):
    M = len(y_pred)
    M_ = 0
    res = 0
    for i in range(M):
        if sum(y_true[i]) != 0:
            res += sum(y_pred[i]*y_true[i]) / sum(y_true[i])
            M_ += 1
    if M_ == 0:
        return 0
    return res / M_

In [2]:
import pandas as pd
from keras.preprocessing import image

labels_list_file = r"/home/anhaoran/data/pedestrian_attributes_PETA/PETA/labels.txt" 
labels_list_data = open(labels_list_file)
lines = labels_list_data.readlines()
attributes_list = []
for line in lines:
    data = line.split()
    #print(attr)
    attributes_list.append(data[1])
print("The total attributes labeled: ", len(attributes_list))
print(attributes_list)

filename = r"../results/PETA.csv"
data = np.array(pd.read_csv(filename))[:, 1:]
length = len(data)
data_x = np.zeros((length, 160, 75, 3))
data_y = np.zeros((length, 61))
for i in range(length):
    #img = image.load_img(path + m)
    img = image.load_img(data[i, 0], target_size=(160, 75, 3))
    data_x[i] = image.img_to_array(img)
    data_y[i] = np.array(data[i, 1:1+61], dtype="float32")
X_test = data_x[11400:]
y_test = data_y[11400:]
print("The shape of the X_test is: ", X_test.shape)
print("The shape of the y_test is: ", y_test.shape)

Using TensorFlow backend.


The total attributes labeled:  65
['accessoryHeadphone', 'personalLess15', 'personalLess30', 'personalLess45', 'personalLess60', 'personalLarger60', 'carryingBabyBuggy', 'carryingBackpack', 'hairBald', 'footwearBoots', 'lowerBodyCapri', 'carryingOther', 'carryingShoppingTro', 'carryingUmbrella', 'lowerBodyCasual', 'upperBodyCasual', 'personalFemale', 'carryingFolder', 'lowerBodyFormal', 'upperBodyFormal', 'accessoryHairBand', 'accessoryHat', 'lowerBodyHotPants', 'upperBodyJacket', 'lowerBodyJeans', 'accessoryKerchief', 'footwearLeatherShoes', 'upperBodyLogo', 'hairLong', 'lowerBodyLongSkirt', 'upperBodyLongSleeve', 'lowerBodyPlaid', 'lowerBodyThinStripes', 'carryingLuggageCase', 'personalMale', 'carryingMessengerBag', 'accessoryMuffler', 'accessoryNothing', 'carryingNothing', 'upperBodyNoSleeve', 'upperBodyPlaid', 'carryingPlasticBags', 'footwearSandals', 'footwearShoes', 'hairShort', 'lowerBodyShorts', 'upperBodyShortSleeve', 'lowerBodyShortSkirt', 'footwearSneakers', 'footwearStockin

# ALL_selected

In [5]:
selected = [27,32,50,56,7,11,21,23,24,26,28,29,35,36,37,38,41,42,43,45,46,47,48,54,57,58,59,60,2,3,4,5,14,15,18,19,31,34,40]
for i in selected:
    print(attributes_list[i], end=",")
print()
mA_val = []
acc_val = []
prec_val = []
rec_val = []
f1_val = []
for i in range(1, 9):
    ###GoogLeNet_depth1
    predictions_prob = np.load("../results/predictions/GoogLeNet_depth"+str(i)+"_binary61_predictions50_imagenet_test7600.npy")
    predictions = np.array(predictions_prob >= 0.5, dtype="float64")[:, selected]
    label = y_test[:, selected]
    """
    print("GoogLeNet_depth1")
    print(mA(predictions, label))
    print(acc(predictions, label))
    prec_value = prec(predictions, label)
    print(prec_value)
    rec_value = rec(predictions, label)
    print(rec_value)
    print(2*prec_value*rec_value/(prec_value+rec_value))
    """
    mA_val.append(mA(predictions, label))
    acc_val.append(acc(predictions, label))
    prec_value = prec(predictions, label)
    prec_val.append(prec_value)
    rec_value = rec(predictions, label)
    rec_val.append(rec_value)
    f1_val.append(2*prec_value*rec_value/(prec_value+rec_value))
predictions_prob = np.load("../results/predictions/GoogLeNet_binary61_predictions50_imagenet_test7600.npy")
predictions = np.array(predictions_prob >= 0.5, dtype="float64")[:, selected]
label = y_test[:, selected]
mA_val.append(mA(predictions, label))
acc_val.append(acc(predictions, label))
prec_value = prec(predictions, label)
prec_val.append(prec_value)
rec_value = rec(predictions, label)
rec_val.append(rec_value)
f1_val.append(2*prec_value*rec_value/(prec_value+rec_value))

upperBodyLogo,lowerBodyThinStripes,upperBodyThinStripes,upperBodyThickStripes,carryingBackpack,carryingOther,accessoryHat,upperBodyJacket,lowerBodyJeans,footwearLeatherShoes,hairLong,lowerBodyLongSkirt,carryingMessengerBag,accessoryMuffler,accessoryNothing,carryingNothing,carryingPlasticBags,footwearSandals,footwearShoes,lowerBodyShorts,upperBodyShortSleeve,lowerBodyShortSkirt,footwearSneakers,accessorySunglasses,lowerBodyTrousers,upperBodyTshirt,upperBodyOther,upperBodyVNeck,personalLess30,personalLess45,personalLess60,personalLarger60,lowerBodyCasual,upperBodyCasual,lowerBodyFormal,upperBodyFormal,lowerBodyPlaid,personalMale,upperBodyPlaid,


In [10]:
for i in range(9):
    print(mA_val[i])

0.5727893673767459
0.6441119908873184
0.6732497872130678
0.6991313715714588
0.7248546205044677
0.7612663590148997
0.7953555858472662
0.8220375123019488
0.7966955022923461


# Low_selected

In [11]:
selected = [27,32,50,56,7,11,21,23,24,26,28,29,35,36,37,38,41,42,43,45,46,47,48,54,57,58,59,60,2,3,4,5,14,15,18,19,31,34,40]
for i in range(4):
    print(attributes_list[selected[i]], end=",")
print()
mA_val = []
acc_val = []
prec_val = []
rec_val = []
f1_val = []
for i in range(1, 9):
    ###GoogLeNet_depth1
    predictions_prob = np.load("../results/predictions/GoogLeNet_depth"+str(i)+"_binary61_predictions50_imagenet_test7600.npy")
    predictions = np.array(predictions_prob >= 0.5, dtype="float64")[:, selected]
    predictions = predictions[:, :4]
    label = y_test[:, selected]
    label = label[:, :4]
    """
    print("GoogLeNet_depth1")
    print(mA(predictions, label))
    print(acc(predictions, label))
    prec_value = prec(predictions, label)
    print(prec_value)
    rec_value = rec(predictions, label)
    print(rec_value)
    print(2*prec_value*rec_value/(prec_value+rec_value))
    """
    mA_val.append(mA(predictions, label))
    acc_val.append(acc(predictions, label))
    prec_value = prec(predictions, label)
    prec_val.append(prec_value)
    rec_value = rec(predictions, label)
    rec_val.append(rec_value)
    f1_val.append(2*prec_value*rec_value/(prec_value+rec_value))
predictions_prob = np.load("../results/predictions/GoogLeNet_binary61_predictions50_imagenet_test7600.npy")
predictions = np.array(predictions_prob >= 0.5, dtype="float64")[:, selected]
predictions = predictions[:, :4]
label = y_test[:, selected]
label = label[:, :4]
mA_val.append(mA(predictions, label))
acc_val.append(acc(predictions, label))
prec_value = prec(predictions, label)
prec_val.append(prec_value)
rec_value = rec(predictions, label)
rec_val.append(rec_value)
f1_val.append(2*prec_value*rec_value/(prec_value+rec_value))

upperBodyLogo,lowerBodyThinStripes,upperBodyThinStripes,upperBodyThickStripes,


In [16]:
for i in range(9):
    print(mA_val[i])

0.5032894736842105
0.585293887995433
0.569088563120074
0.5934447870369368
0.5925341831624173
0.6167058040124369
0.658434688723863
0.6791903953755694
0.6325996044922395


# Mid_selected

In [17]:
selected = [27,32,50,56,7,11,21,23,24,26,28,29,35,36,37,38,41,42,43,45,46,47,48,54,57,58,59,60,2,3,4,5,14,15,18,19,31,34,40]
for i in range(4, 28):
    print(attributes_list[selected[i]], end=",")
print()
mA_val = []
acc_val = []
prec_val = []
rec_val = []
f1_val = []
for i in range(1, 9):
    ###GoogLeNet_depth1
    predictions_prob = np.load("../results/predictions/GoogLeNet_depth"+str(i)+"_binary61_predictions50_imagenet_test7600.npy")
    predictions = np.array(predictions_prob >= 0.5, dtype="float64")[:, selected]
    predictions = predictions[:, 4:28]
    label = y_test[:, selected]
    label = label[:, 4:28]
    """
    print("GoogLeNet_depth1")
    print(mA(predictions, label))
    print(acc(predictions, label))
    prec_value = prec(predictions, label)
    print(prec_value)
    rec_value = rec(predictions, label)
    print(rec_value)
    print(2*prec_value*rec_value/(prec_value+rec_value))
    """
    mA_val.append(mA(predictions, label))
    acc_val.append(acc(predictions, label))
    prec_value = prec(predictions, label)
    prec_val.append(prec_value)
    rec_value = rec(predictions, label)
    rec_val.append(rec_value)
    f1_val.append(2*prec_value*rec_value/(prec_value+rec_value))
predictions_prob = np.load("../results/predictions/GoogLeNet_binary61_predictions50_imagenet_test7600.npy")
predictions = np.array(predictions_prob >= 0.5, dtype="float64")[:, selected]
predictions = predictions[:, 4:28]
label = y_test[:, selected]
label = label[:, 4:28]
mA_val.append(mA(predictions, label))
acc_val.append(acc(predictions, label))
prec_value = prec(predictions, label)
prec_val.append(prec_value)
rec_value = rec(predictions, label)
rec_val.append(rec_value)
f1_val.append(2*prec_value*rec_value/(prec_value+rec_value))

carryingBackpack,carryingOther,accessoryHat,upperBodyJacket,lowerBodyJeans,footwearLeatherShoes,hairLong,lowerBodyLongSkirt,carryingMessengerBag,accessoryMuffler,accessoryNothing,carryingNothing,carryingPlasticBags,footwearSandals,footwearShoes,lowerBodyShorts,upperBodyShortSleeve,lowerBodyShortSkirt,footwearSneakers,accessorySunglasses,lowerBodyTrousers,upperBodyTshirt,upperBodyOther,upperBodyVNeck,


In [22]:
for i in range(9):
    print(mA_val[i])

0.5836008028623726
0.6520495570813896
0.6925999839252683
0.721578672496905
0.739422457375012
0.7850898341579856
0.8115070872882207
0.8361444806008723
0.8173555192517554


# High_selected

In [23]:
selected = [27,32,50,56,7,11,21,23,24,26,28,29,35,36,37,38,41,42,43,45,46,47,48,54,57,58,59,60,2,3,4,5,14,15,18,19,31,34,40]
for i in range(28, len(selected)):
    print(attributes_list[selected[i]], end=",")
print()
mA_val = []
acc_val = []
prec_val = []
rec_val = []
f1_val = []
for i in range(1, 9):
    ###GoogLeNet_depth1
    predictions_prob = np.load("../results/predictions/GoogLeNet_depth"+str(i)+"_binary61_predictions50_imagenet_test7600.npy")
    predictions = np.array(predictions_prob >= 0.5, dtype="float64")[:, selected]
    predictions = predictions[:, 28:]
    label = y_test[:, selected]
    label = label[:, 28:]
    """
    print("GoogLeNet_depth1")
    print(mA(predictions, label))
    print(acc(predictions, label))
    prec_value = prec(predictions, label)
    print(prec_value)
    rec_value = rec(predictions, label)
    print(rec_value)
    print(2*prec_value*rec_value/(prec_value+rec_value))
    """
    mA_val.append(mA(predictions, label))
    acc_val.append(acc(predictions, label))
    prec_value = prec(predictions, label)
    prec_val.append(prec_value)
    rec_value = rec(predictions, label)
    rec_val.append(rec_value)
    f1_val.append(2*prec_value*rec_value/(prec_value+rec_value))
predictions_prob = np.load("../results/predictions/GoogLeNet_binary61_predictions50_imagenet_test7600.npy")
predictions = np.array(predictions_prob >= 0.5, dtype="float64")[:, selected]
predictions = predictions[:, 28:]
label = y_test[:, selected]
label = label[:, 28:]
mA_val.append(mA(predictions, label))
acc_val.append(acc(predictions, label))
prec_value = prec(predictions, label)
prec_val.append(prec_value)
rec_value = rec(predictions, label)
rec_val.append(rec_value)
f1_val.append(2*prec_value*rec_value/(prec_value+rec_value))

personalLess30,personalLess45,personalLess60,personalLarger60,lowerBodyCasual,upperBodyCasual,lowerBodyFormal,upperBodyFormal,lowerBodyPlaid,personalMale,upperBodyPlaid,


In [28]:
for i in range(9):
    print(mA_val[i])

0.574473469478119
0.6481820656973037
0.6689079849657188
0.6885869275648567
0.7411867718203883
0.7618553423399714
0.8099053634755122
0.8432030785320715
0.8112903372173093
